In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import os

import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from keras.preprocessing import image
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

ValueError: tf.enable_eager_execution must be called at program startup.

In [2]:

path= "D:\\Learn\\大三下\\summer\\ChromosomesDataSet"

os.chdir(path)
os.listdir(path)

['9total.zip',
 '9total_x.npy',
 '9total_y.npy',
 '9左右全部npy',
 '9左右全部npy.zip',
 'chromosome_l_x_test.npy',
 'chromosome_l_x_train.npy',
 'chromosome_l_y_test.npy',
 'chromosome_l_y_train.npy',
 'chromosome_r_x_test.npy',
 'chromosome_r_x_train.npy',
 'chromosome_r_y_test.npy',
 'chromosome_r_y_train.npy',
 'HardData_x.npy',
 'HardData_y.npy',
 'my_model.h5']

In [22]:
x_train_savepath = './9total_x.npy'
y_train_savepath = './9total_y.npy'

x_test_savepath = './HardData_x.npy'
y_test_savepath = './HardData_y.npy'
print('-------------Load Datasets-----------------')
x_train_save = np.load(x_train_savepath)
print(x_train_save.shape)
y_train = np.load(y_train_savepath)
x_test_save = np.load(x_test_savepath)
y_test = np.load(y_test_savepath)
x_train = np.reshape(x_train_save, (len(x_train_save), 150, 150,1))
x_test = np.reshape(x_test_save, (len(x_test_save), 150, 150,1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

-------------Load Datasets-----------------
(2352, 22500)
(2352, 150, 150, 1)
(289, 150, 150, 1)


In [7]:
class Autoencoder():
    def __init__(self):

        self.img_shape = (150,150,1)

        optimizer = Adam(lr=0.001)

        self.autoencoder_model = self.build_model()
        self.autoencoder_model.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.autoencoder_model.summary()

    def build_model(self):
        input_layer = Input(shape=self.img_shape)

        # encoder
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
        h = MaxPooling2D((2, 2), padding='same')(h)
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(h)
        h = MaxPooling2D((3, 3), padding='same')(h)

        # decoder
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(h)
        h = UpSampling2D((3, 3))(h)
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(h)
        h = UpSampling2D((2, 2))(h)

        output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(h)

        return Model(input_layer, output_layer)

    def train_model(self, x_train, y_train, x_test, y_test, epochs, batch_size):
        early_stopping = EarlyStopping(monitor='val_loss',
                                       min_delta=0,
                                       patience=5,
                                       verbose=1,
                                       mode='auto')
        history = self.autoencoder_model.fit(x_train, x_train,
                                             batch_size=batch_size,
                                             epochs=epochs,
                                             validation_data=(x_test, x_test),
                                             callbacks=[early_stopping])
        
        file = self.autoencoder_model.save_weights('my_model.h5')

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()

    def eval_model(self, x_test):
        preds = self.autoencoder_model.predict(x_test)
        return preds


In [6]:
ae = Autoencoder()
ae.train_model(x_train, x_train, x_test, x_test, epochs=1, batch_size=20)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        36928     
______________________________

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [8]:
decoded_test = ae.eval_model(x_test)

import matplotlib.pyplot as plt
%matplotlib inline
shape = (150, 150)
fig, axes = plt.subplots(2,10,
                         figsize=(10, 2),
                         subplot_kw={
                             'xticks': [],
                             'yticks': []
                         },
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    axes[0][i].imshow(np.reshape(x_test[i], shape),cmap='gray')
    axes[1][i].imshow(np.reshape(decoded_test[i], shape),cmap='gray')
plt.show()

RuntimeError: `get_session` is not available when TensorFlow is executing eagerly.

In [53]:

from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler=MinMaxScaler(
)
scaled_input_test=scaler.fit_transform(x_test.reshape(-1,22500))
scaled_output_test=scaler.fit_transform(decoded_test.reshape(-1,22500))

decoded_train = ae.eval_model(x_train)
scaler=MinMaxScaler(
)
scaled_input_train=scaler.fit_transform(x_train.reshape(-1,22500))
scaled_output_train=scaler.fit_transform(decoded_train.reshape(-1,22500))
import pandas as pd
from keras import losses
x = losses.binary_crossentropy(scaled_input_train,scaled_output_train)
y = losses.binary_crossentropy(scaled_input_test,scaled_output_test)


ValueError: tf.enable_eager_execution must be called at program startup.

In [52]:
import pandas as pd

sequences=range(1,len(x_train)+len(y_train)+1)
seqs_ds = pd.DataFrame(sequences)
mse = np.append(x,y)
seqs_ds['MSE'] = mse
seqs_d

(2352,)


In [29]:

print(x_train.shape)
print(x_test.shape)

(2352, 150, 150, 1)
(289, 150, 150, 1)


In [37]:
xxx=np.vstack((x_train,x_test))

In [38]:
print(xxx.shape)


(2641, 150, 150, 1)
2641


In [43]:
print(y_train.shape)
print(y_test.shape)
yyy=np.append(y_train,y_test)
print(yyy.shape)

(2352,)
(289,)
(2641,)


In [44]:
xxx=np.vstack((x_train,x_test))
yyy=np.append(y_train,y_test)
from sklearn.model_selection import train_test_split
data, labels = xxx,yyy
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.20)


In [49]:
print(data_train.shape)

(2112, 150, 150, 1)
